<a href="https://colab.research.google.com/github/loet8/s2025-assignment1-basics/blob/main/Training_TS_and_OWT_ECE496B_Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!git clone https://github.com/loet8/s2025-assignment1-basics

Cloning into 's2025-assignment1-basics'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 225 (delta 43), reused 39 (delta 39), pack-reused 173 (from 1)
Receiving objects: 100% (225/225), 11.89 MiB | 11.19 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [23]:
%cd s2025-assignment1-basics/

/content/s2025-assignment1-basics/s2025-assignment1-basics


In [5]:
!git pull

Already up to date.


In [6]:
!git checkout main

Already on 'main'
Your branch is up to date with 'origin/main'.


No need for Conda environment on Colab

In [24]:
!pip install -e .'[test]'

Obtaining file:///content/s2025-assignment1-basics/s2025-assignment1-basics
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ece496b_basics
    Found existing installation: ece496b_basics 0.1.7.dev0
    Uninstalling ece496b_basics-0.1.7.dev0:
      Successfully uninstalled ece496b_basics-0.1.7.dev0
  Running setup.py develop for ece496b_basics


In [25]:
!pytest tests/test_train_bpe.py

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0
rootdir: /content/s2025-assignment1-basics/s2025-assignment1-basics
configfile: pytest.ini
plugins: langsmith-0.3.8, anyio-3.7.1, typeguard-4.4.1
collected 2 items                                                                                  

tests/test_train_bpe.py::test_train_bpe_speed PASSED                                         [ 50%]
tests/test_train_bpe.py::test_train_bpe PASSED                                               [100%]

======================================== 2 passed in 3.95s =========================================


In [ ]:
!pytest

In [26]:
%cd ..
%mkdir -p data
%cd data


/content/s2025-assignment1-basics
/content/s2025-assignment1-basics/data


In [27]:


!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip owt_valid.txt.gz

%cd ..
%cd s2025-assignment1-basics/

--2025-02-16 07:26:41--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
Resolving huggingface.co (huggingface.co)... 3.165.160.11, 3.165.160.12, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.11|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/42/7f/427f7497b6c6596c18b46d5a72e61364fcad12aa433c60a0dbd4d344477b9d81/6418d412de72888f52b5142c761ac21a582f7d1166f0bfbdb5f03ccfdec90443?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyStoriesV2-GPT4-train.txt%3B+filename%3D%22TinyStoriesV2-GPT4-train.txt%22%3B&response-content-type=text%2Fplain&Expires=1739694401&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTY5NDQwMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Mi83Zi80MjdmNzQ5N2I2YzY1OTZjMThiNDZkNWE3MmU2MTM2NGZjYWQxMmFhNDMzYzYwYTBkYmQ0ZDM0NDQ3N2I5ZDgxLzY0MThkNDEyZGU3Mjg4OGY1MmI1MTQyYzc2M

In [29]:
%cd ..
!ls



/content/s2025-assignment1-basics
adapters.py				 ece496b_basics.egg-info  requirements-test.txt
CHANGELOG.md				 learning_rate_tuning.py  requirements.txt
cs336_spring2024_assignment1_basics.pdf  __pycache__		  s2025-assignment1-basics
data					 pytest.ini		  setup.py
ece496b_basics				 README.md		  tests


In [35]:
import os
import time
import tempfile
import  adapters

REPO_DIR = "/content/s2025-assignment1-basics"
DATA_DIR = os.path.join(REPO_DIR, "data")  # Define DATA_DIR here

TINYSTORIES_VOCAB_PATH = os.path.join(DATA_DIR, "tinystories_bpe_vocab.txt")
TINYSTORIES_MERGES_PATH = os.path.join(DATA_DIR, "tinystories_bpe_merges.txt")
MERGES_PATH = os.path.join(DATA_DIR, "bpe_merges.txt")
INPUT_PATH = os.path.join(DATA_DIR, "TinyStoriesV2-GPT4-train.txt")

VOCAB_SIZE = 10000
SPECIAL_TOKENS = ["<|endoftext|>"]
BATCH_SIZE = 10000
MAX_LINES = 100000

def train_bpe_streaming(input_path, vocab_size, special_tokens, batch_size, max_lines):
    print("Starting streaming BPE training...")
    vocab, merges = None, []

    with open(input_path, "r", encoding="utf-8") as file:
        line_count = 0
        batch_data = []

        for line in file:
            batch_data.append(line)
            line_count += 1

            if line_count % batch_size == 0:
                print(f"Processing batch {line_count // batch_size}...")

                with tempfile.NamedTemporaryFile(mode="w+", delete=False) as tmp_file:
                    tmp_file.writelines(batch_data)
                    tmp_file.flush()
                    batch_data.clear()

                    vocab, merges = adapters.run_train_bpe(tmp_file.name, vocab_size, special_tokens)

                os.remove(tmp_file.name)

            if line_count >= max_lines:
                break

    print("Training complete!")
    return vocab, merges

start_time = time.time()
vocab, merges = train_bpe_streaming(INPUT_PATH, VOCAB_SIZE, SPECIAL_TOKENS, BATCH_SIZE, MAX_LINES)
elapsed_time = time.time() - start_time

print(f"Saving vocabulary and merges... (Elapsed Time: {elapsed_time:.2f}s)")
with open(TINYSTORIES_VOCAB_PATH, "w", encoding="utf-8") as vf:
    for k, v in vocab.items():
        vf.write(f"{k} {v.hex()}\n")

with open(TINYSTORIES_MERGES_PATH, "w", encoding="utf-8") as mf:
    for merge in merges:
        mf.write(f"{merge[0].hex()} {merge[1].hex()}\n")


print(f"Saved vocabulary to {TINYSTORIES_VOCAB_PATH}")
print(f"Saved merges to {TINYSTORIES_MERGES_PATH}\n")

longest_token = max(vocab.values(), key=len)
longest_token_id = [k for k, v in vocab.items() if v == longest_token][0]

print(f"Longest Token: {longest_token.decode('utf-8', errors='ignore')}")
print(f"Token ID: {longest_token_id}")
print(f"Length: {len(longest_token)} bytes")


print("Loading trained tokenizer...")
ts_tokenizer = adapters.get_tokenizer(vocab, merges, SPECIAL_TOKENS)

sample_text = "Hello world, how are you?"
encoded = ts_tokenizer.encode(sample_text)
decoded = ts_tokenizer.decode(encoded)

print(f"\nEncoded: {encoded}")
print(f"Decoded: {decoded}")

if sample_text == decoded:
    print("Encoding-Decoding Test Passed!")
else:
    print("Encoding-Decoding Mismatch!")


Starting streaming BPE training...
Processing batch 1...
Processing batch 2...
Processing batch 3...
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Processing batch 9...
Processing batch 10...
Training complete!
Saving vocabulary and merges... (Elapsed Time: 53.49s)
Saved vocabulary to /content/s2025-assignment1-basics/data/tinystories_bpe_vocab.txt
Saved merges to /content/s2025-assignment1-basics/data/tinystories_bpe_merges.txt

Longest Token:  enthusiastically
Token ID: 7559
Length: 17 bytes
Loading trained tokenizer...

Encoded: [1177, 1520, 44, 769, 486, 348, 63]
Decoded: Hello world, how are you?
Encoding-Decoding Test Passed!


In [36]:
import os
import time
import tempfile
import adapters

REPO_DIR = "/content/s2025-assignment1-basics"
DATA_DIR = os.path.join(REPO_DIR, "data")
VOCAB_PATH = os.path.join(DATA_DIR, "owt_bpe_vocab.txt")
MERGES_PATH = os.path.join(DATA_DIR, "owt_bpe_merges.txt")
INPUT_PATH = os.path.join(DATA_DIR, "owt_train.txt")


VOCAB_SIZE = 32000
SPECIAL_TOKENS = ["<|endoftext|>"]
BATCH_SIZE = 10000
MAX_LINES = 100000

def train_bpe_streaming(input_path, vocab_size, special_tokens, batch_size, max_lines):
    print("Starting streaming BPE training...")
    vocab, merges = None, []

    with open(input_path, "r", encoding="utf-8") as file:
        line_count = 0
        batch_data = []

        for line in file:
            batch_data.append(line)
            line_count += 1

            if line_count % batch_size == 0:
                print(f"Processing batch {line_count // batch_size}...")

                with tempfile.NamedTemporaryFile(mode="w+", delete=False) as tmp_file:
                    tmp_file.writelines(batch_data)
                    tmp_file.flush()
                    batch_data.clear()

                    vocab, merges = adapters.run_train_bpe(tmp_file.name, vocab_size, special_tokens)

                os.remove(tmp_file.name)

            if line_count >= max_lines:
                break

    print("Training complete!")
    return vocab, merges

start_time = time.time()
vocab, merges = train_bpe_streaming(INPUT_PATH, VOCAB_SIZE, SPECIAL_TOKENS, BATCH_SIZE, MAX_LINES)
elapsed_time = time.time() - start_time

print(f"Saving vocabulary and merges... (Elapsed Time: {elapsed_time:.2f}s)")
with open(VOCAB_PATH, "w", encoding="utf-8") as vf:
    for k, v in vocab.items():
        vf.write(f"{k} {v.hex()}\n")

with open(MERGES_PATH, "w", encoding="utf-8") as mf:
    for merge in merges:
        mf.write(f"{merge[0].hex()} {merge[1].hex()}\n")

print(f"Saved vocabulary to {VOCAB_PATH}")
print(f"Saved merges to {MERGES_PATH}\n")


longest_token = max(vocab.values(), key=len)
longest_token_id = [k for k, v in vocab.items() if v == longest_token][0]

print(f"Longest Token: {longest_token.decode('utf-8', errors='ignore')}")
print(f"Token ID: {longest_token_id}")
print(f"Length: {len(longest_token)} bytes")


print("Loading trained tokenizer...")
owt_tokenizer = adapters.get_tokenizer(vocab, merges, SPECIAL_TOKENS)

sample_text = "Hello world, how are you?"
encoded = owt_tokenizer.encode(sample_text)
decoded = owt_tokenizer.decode(encoded)

print(f"\nEncoded: {encoded}")
print(f"Decoded: {decoded}")

if sample_text == decoded:
    print("Encoding-Decoding Test Passed!")
else:
    print("Encoding-Decoding Mismatch!")


Starting streaming BPE training...
Processing batch 1...
Processing batch 2...
Processing batch 3...
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Processing batch 9...
Processing batch 10...
Training complete!
Saving vocabulary and merges... (Elapsed Time: 1347.34s)
Saved vocabulary to /content/s2025-assignment1-basics/data/owt_bpe_vocab.txt
Saved merges to /content/s2025-assignment1-basics/data/owt_bpe_merges.txt

Longest Token: ______________________________________________________
Token ID: 26683
Length: 54 bytes
Loading trained tokenizer...

Encoded: [17936, 872, 44, 742, 373, 346, 63]
Decoded: Hello world, how are you?
Encoding-Decoding Test Passed!


In [37]:
!ls -lh /content/s2025-assignment1-basics/data/


total 14G
-rw-r--r-- 1 root root 477K Feb 16 08:04 owt_bpe_merges.txt
-rw-r--r-- 1 root root 623K Feb 16 08:04 owt_bpe_vocab.txt
-rw-r--r-- 1 root root  12G Apr  1  2024 owt_train.txt
-rw-r--r-- 1 root root 277M Apr  1  2024 owt_valid.txt
-rw-r--r-- 1 root root 104K Feb 16 07:35 tinystories_bpe_merges.txt
-rw-r--r-- 1 root root 135K Feb 16 07:35 tinystories_bpe_vocab.txt
-rw-r--r-- 1 root root 2.1G May 19  2023 TinyStoriesV2-GPT4-train.txt
-rw-r--r-- 1 root root  22M May 19  2023 TinyStoriesV2-GPT4-valid.txt


In [43]:
import os
import adapters

REPO_DIR = "/content/s2025-assignment1-basics"
DATA_DIR = os.path.join(REPO_DIR, "data")

TINYSTORIES_VOCAB_PATH = os.path.join(DATA_DIR, "tinystories_bpe_vocab.txt")
TINYSTORIES_MERGES_PATH = os.path.join(DATA_DIR, "tinystories_bpe_merges.txt")

OWT_VOCAB_PATH = os.path.join(DATA_DIR, "owt_bpe_vocab.txt")
OWT_MERGES_PATH = os.path.join(DATA_DIR, "owt_bpe_merges.txt")

SPECIAL_TOKENS = ["<|endoftext|>"]

def load_tokenizer(vocab_path, merges_path, special_tokens):
    """Loads a tokenizer from vocabulary and merges files."""
    print(f"Loading tokenizer from {vocab_path} and {merges_path}...")

    try:
        with open(vocab_path, "r", encoding="utf-8") as vf:
            vocab = {int(k): bytes.fromhex(v) for k, v in (line.strip().split() for line in vf)}

        with open(merges_path, "r", encoding="utf-8") as mf:
            merges = [tuple(bytes.fromhex(m) for m in line.strip().split()) for line in mf]

        tokenizer = adapters.get_tokenizer(vocab, merges, special_tokens)
        print(f"Tokenizer loaded successfully from {vocab_path}")

        return tokenizer

    except FileNotFoundError:
        print(f"Error: Missing tokenizer files: {vocab_path} or {merges_path}")
        return None

    except Exception as e:
        print(f"Unexpected error while loading tokenizer: {e}")
        return None

tiny_tokenizer = load_tokenizer(TINYSTORIES_VOCAB_PATH, TINYSTORIES_MERGES_PATH, SPECIAL_TOKENS)

owt_tokenizer = load_tokenizer(OWT_VOCAB_PATH, OWT_MERGES_PATH, SPECIAL_TOKENS)

if tiny_tokenizer and owt_tokenizer:
    print("Tokenizers loaded successfully.")
else:
    print("Tokenizer loading failed.")


Loading tokenizer from /content/s2025-assignment1-basics/data/tinystories_bpe_vocab.txt and /content/s2025-assignment1-basics/data/tinystories_bpe_merges.txt...
Tokenizer loaded successfully from /content/s2025-assignment1-basics/data/tinystories_bpe_vocab.txt
Loading tokenizer from /content/s2025-assignment1-basics/data/owt_bpe_vocab.txt and /content/s2025-assignment1-basics/data/owt_bpe_merges.txt...
Tokenizer loaded successfully from /content/s2025-assignment1-basics/data/owt_bpe_vocab.txt
Tokenizers loaded successfully.


In [44]:
import os
import random

def fast_sample_lines(file_path, num_samples=10):
    file_size = os.path.getsize(file_path)
    samples = set()

    with open(file_path, "r", encoding="utf-8", errors="replace") as f:
        while len(samples) < num_samples:
            f.seek(random.randint(0, file_size - 2000))
            f.readline()
            line = f.readline().strip()
            if line:
                samples.add(line)

    return list(samples)

TINYSTORIES_DATA_PATH = "/content/s2025-assignment1-basics/data/TinyStoriesV2-GPT4-train.txt"
OWT_DATA_PATH = "/content/s2025-assignment1-basics/data/owt_train.txt"

tiny_samples = fast_sample_lines(TINYSTORIES_DATA_PATH, 10)
owt_samples = fast_sample_lines(OWT_DATA_PATH, 10)

print(" TinyStories Sample:")
for i, line in enumerate(tiny_samples[:3]):
    print(f"{i+1}: {line}")

print("\n OpenWebText Sample:")
for i, line in enumerate(owt_samples[:3]):
    print(f"{i+1}: {line}")


 TinyStories Sample:
1: The bird was very thankful to Sam and Fred. They all became friends and learned new things every day. They played and had fun in the quiet forest, and they were always there to help each other.
2: Then, something unexpected happened. A big wind came and blew the doll and the car up in the air. The toys landed on the other side of the fence. Now, Lily had Tim's car and Tim had Lily's doll. They both looked at the toys and then at each other. They realized that they could play with the new toys and have fun. So, they decided to share their toys and play together. The stubborn little girl and boy became good friends, and they always shared their toys from that day on.
3: As they walked, Clara's mommy said "These are very expensive, Clara. We don't have enough money to buy them."

 OpenWebText Sample:
1: He said that abortion isn’t the main issue at this point, but rather what he sees as a general reluctance to take on issues in the House of Commons.
2: Nah, fuck yo

In [45]:
import numpy as np

def compute_compression_ratio(samples, tokenizer):
    total_bytes = sum(len(line.encode("utf-8")) for line in samples)
    tokenized_samples = [tokenizer.encode(line) for line in samples]
    total_tokens = sum(len(tokens) for tokens in tokenized_samples)

    return total_bytes / total_tokens

tiny_ratio = compute_compression_ratio(tiny_samples, tiny_tokenizer)
owt_ratio = compute_compression_ratio(owt_samples, owt_tokenizer)

print(f"TinyStories Tokenizer Compression Ratio: {tiny_ratio:.2f} bytes/token")
print(f"OpenWebText Tokenizer Compression Ratio: {owt_ratio:.2f} bytes/token")


TinyStories Tokenizer Compression Ratio: 4.19 bytes/token
OpenWebText Tokenizer Compression Ratio: 4.38 bytes/token


In [55]:
print("OpenWebText tokenized with its own tokenizer:", owt_tokenizer.encode(owt_samples[0]))
print("OpenWebText tokenized with TinyStories tokenizer:", tiny_tokenizer.encode(owt_samples[0]))



owt_with_tiny_ratio = compute_compression_ratio(owt_samples, tiny_tokenizer)

print(f"Compression Ratio of OpenWebText with TinyStories Tokenizer: {owt_with_tiny_ratio:.2f} bytes/token")


OpenWebText tokenized with its own tokenizer: [1083, 482, 326, 20442, 1866, 315, 116, 262, 1281, 1919, 352, 429, 1248, 44, 479, 2252, 601, 340, 8231, 360, 258, 3152, 961, 9707, 577, 283, 1061, 324, 2792, 286, 262, 1979, 287, 7259, 46]
OpenWebText tokenized with TinyStories tokenizer: [957, 323, 387, 689, 903, 846, 4294, 1271, 116, 263, 281, 516, 424, 115, 431, 448, 721, 3336, 44, 420, 7266, 564, 285, 1363, 457, 259, 299, 301, 283, 440, 1940, 6362, 1534, 266, 749, 354, 424, 115, 5542, 315, 263, 311, 544, 373, 815, 284, 109, 1754, 46]
Compression Ratio of OpenWebText with TinyStories Tokenizer: 3.00 bytes/token


In [47]:
import time

def measure_tokenizer_throughput(tokenizer, text_samples):
    total_bytes = sum(len(line.encode("utf-8")) for line in text_samples)

    start_time = time.time()
    _ = [tokenizer.encode(line) for line in text_samples]
    elapsed_time = time.time() - start_time

    throughput = total_bytes / elapsed_time
    return throughput

tiny_throughput = measure_tokenizer_throughput(tiny_tokenizer, tiny_samples)
owt_throughput = measure_tokenizer_throughput(owt_tokenizer, owt_samples)

pile_size_bytes = 825 * 1024**3
tiny_estimated_time = pile_size_bytes / tiny_throughput
owt_estimated_time = pile_size_bytes / owt_throughput

print(f"TinyStories Tokenizer Throughput: {tiny_throughput:.2f} bytes/sec")
print(f"OpenWebText Tokenizer Throughput: {owt_throughput:.2f} bytes/sec")
print(f"Estimated time to tokenize the Pile with TinyStories: {tiny_estimated_time / 3600:.2f} hours")
print(f"Estimated time to tokenize the Pile with OpenWebText: {owt_estimated_time / 3600:.2f} hours")


TinyStories Tokenizer Throughput: 24307.57 bytes/sec
OpenWebText Tokenizer Throughput: 19938.01 bytes/sec
Estimated time to tokenize the Pile with TinyStories: 10123.01 hours
Estimated time to tokenize the Pile with OpenWebText: 12341.54 hours


In [ ]:
import numpy as np

def tokenize(input_path, tokenizer, output_path, batch_size=10000):
    tokenized_data = []

    with open(input_path, "r", encoding="utf-8", errors="replace") as file:
        batch = []
        for line in file:
            batch.append(line.strip())

            if len(batch) >= batch_size:
                tokenized_data.extend(map(tokenizer.encode, batch))
                batch = []

        if batch:
            tokenized_data.extend(map(tokenizer.encode, batch))

    tokenized_array = np.concatenate([np.array(tokens, dtype=np.uint16) for tokens in tokenized_data])
    np.save(output_path, tokenized_array)
    print(f"Saved tokenized dataset to {output_path}")

TINYSTORIES_RAW_PATH = "/content/s2025-assignment1-basics/data/TinyStoriesV2-GPT4-train.txt"
OWT_RAW_PATH = "/content/s2025-assignment1-basics/data/owt_train.txt"

TINYSTORIES_TOKENS_PATH = "/content/s2025-assignment1-basics/data/tinystories_tokenized.npy"
OWT_TOKENS_PATH = "/content/s2025-assignment1-basics/data/owt_tokenized.npy"

tokenize(TINYSTORIES_RAW_PATH, tiny_tokenizer, TINYSTORIES_TOKENS_PATH)
tokenize(OWT_RAW_PATH, owt_tokenizer, OWT_TOKENS_PATH)

tiny_loaded = np.load(TINYSTORIES_TOKENS_PATH)
owt_loaded = np.load(OWT_TOKENS_PATH)

print("TinyStories tokenized sample:", tiny_loaded[:10])
print("OpenWebText tokenized sample:", owt_loaded[:10])


In [64]:
import os
import time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import adapters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_random_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

class TokenizedDataset(Dataset):
    def __init__(self, file_path, context_length):
        self.data = np.memmap(file_path, dtype=np.uint16, mode="r")
        self.context_length = context_length

    def __len__(self):
        return len(self.data) - self.context_length

    def __getitem__(self, idx):
        x = torch.from_numpy(self.data[idx: idx + self.context_length]).long()
        y = torch.from_numpy(self.data[idx + 1: idx + self.context_length + 1]).long()
        return x, y

class TransformerLM(nn.Module):
    def __init__(self, vocab_size, context_length, d_model, num_layers, num_heads, d_ff, attn_pdrop, residual_pdrop):
        super().__init__()
        self.vocab_size = vocab_size
        self.context_length = context_length
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.attn_pdrop = attn_pdrop
        self.residual_pdrop = residual_pdrop
        self.weights = self._initialize_weights()

    def _initialize_weights(self):
        return {k: torch.randn(v.shape, device=device, requires_grad=True) for k, v in adapters.run_transformer_lm.__annotations__.items()}

    def forward(self, in_indices):
        return adapters.run_transformer_lm(
            self.vocab_size,
            self.context_length,
            self.d_model,
            self.num_layers,
            self.num_heads,
            self.d_ff,
            self.attn_pdrop,
            self.residual_pdrop,
            self.weights,
            in_indices
        )

def train(model, train_loader, val_loader, optimizer, criterion, epochs, checkpoint_path, log_interval):
    model.to(device)
    best_loss = float("inf")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        start_time = time.time()

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output.view(-1, model.vocab_size), y.reshape(-1))
            loss.backward()

            adapters.run_gradient_clipping(model.parameters(), max_l2_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

            if batch_idx % log_interval == 0:
                print(f"Epoch {epoch+1} | Batch {batch_idx} | Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f} | Time: {time.time() - start_time:.2f}s")

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                output = model(x)
                val_loss += criterion(output.view(-1, model.vocab_size), y.reshape(-1)).item()

        val_loss /= len(val_loader)
        print(f"Validation Loss: {val_loss:.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            adapters.run_save_checkpoint(model, optimizer, epoch, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")


class Args:
    vocab_size = 10000
    context_length = 128
    d_model = 256
    num_layers = 6
    num_heads = 8
    d_ff = 1024
    attn_pdrop = 0.1
    residual_pdrop = 0.1
    batch_size = 64
    epochs = 10
    learning_rate = 3e-4
    log_interval = 10
    train_path = "/content/s2025-assignment1-basics/data/tinystories_tokenized.npy"
    val_path = "/content/s2025-assignment1-basics/data/owt_tokenized.npy"
    checkpoint_path = "model_checkpoint.pth"
    random_seed = 0

args = Args()
set_random_seed(args.random_seed)

train_data = TokenizedDataset(args.train_path, args.context_length)
val_data = TokenizedDataset(args.val_path, args.context_length)

train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)

model = TransformerLM(
    args.vocab_size, args.context_length, args.d_model,
    args.num_layers, args.num_heads, args.d_ff,
    args.attn_pdrop, args.residual_pdrop
).to(device)

optimizer = adapters.AdamWOptimizer(model.parameters(), lr=args.learning_rate)
criterion = adapters.run_cross_entropy

train(model, train_loader, val_loader, optimizer, criterion, args.epochs, args.checkpoint_path, args.log_interval)


FileNotFoundError: [Errno 2] No such file or directory: '/content/s2025-assignment1-basics/data/tinystories_tokenized.npy'

In [ ]:
import torch
import torch.nn.functional as F

def generate_text(
    model, tokenizer, prompt: str, max_tokens: int = 50, temperature: float = 1.0, top_p: float = 0.9, device="cuda"
) -> str:
    model.eval()

    eos_token_id = tokenizer.eos_token_id if hasattr(tokenizer, "eos_token_id") else tokenizer.byte_to_id[b"<|endoftext|>"]

    input_ids = torch.tensor(tokenizer.encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
    generated_tokens = []

    with torch.no_grad():
        for _ in range(max_tokens):
            logits = model(input_ids)[:, -1, :]

            scaled_logits = logits / temperature
            probs = F.softmax(scaled_logits, dim=-1)

            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

            top_p_mask = cumulative_probs > top_p
            top_p_mask[:, 1:] = top_p_mask[:, :-1].clone()
            top_p_mask[:, 0] = False

            sorted_probs[top_p_mask] = 0
            sorted_probs /= sorted_probs.sum(dim=-1, keepdim=True)

            sampled_index = torch.multinomial(sorted_probs, 1)
            next_token = sorted_indices[0, sampled_index].item()

            if next_token == eos_token_id:
                break

            generated_tokens.append(next_token)
            input_ids = torch.cat([input_ids, torch.tensor([[next_token]], dtype=torch.long, device=device)], dim=1)

    return tokenizer.decode(generated_tokens)


